In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, date

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns
import matplotlib.pyplot as plt

chart_color = px.colors.qualitative.Plotly

In [2]:
if os.name =='nt':
    df_path = ''
else:
    df_path = '/Users/admin/_Work/Data/icanKID'

# df_User = pd.read_csv(
#     os.path.join(df_path,'df_User.csv')
#     )
df_User = pd.read_csv(
    os.path.join(df_path,'df_User_Child_DeviceBrand.csv')
    )
df_Engagement = pd.read_csv(
    os.path.join(df_path, 'df_Engagement.csv')
    )
df_Engagement_updated = pd.read_csv(
os.path.join(df_path, 'df_Engagement_updated.csv')
    )
df_Payment = pd.read_csv(
    os.path.join(df_path, 'df_Payment.csv')
    )
df_fake_user = pd.read_csv(
    os.path.join(df_path, 'df_fake_paid_users.csv')
    )
# df_DeviceBrand = pd.read_csv('/Users/admin/_Work/Data/icanKID/df_DeviceBrand.csv')

# Discard fake user
df_User = df_User[~df_User['UserID'].isin(df_fake_user['UserID'])]
df_Engagement = df_Engagement[~df_Engagement['UserID'].isin(df_fake_user['UserID'])]
df_Engagement_updated = df_Engagement_updated[~df_Engagement_updated['UserID'].isin(df_fake_user['UserID'])]
df_Payment = df_Payment[~df_Payment['UserID'].isin(df_fake_user['UserID'])]

In [8]:
def getEvent_before_cutoff(df_events, cutOffDay=3):
    days_from_regis = df_events.groupby('UserID')[['nth_day_from_registration']].min().reset_index()
    days_from_regis.rename(
        columns={
            "nth_day_from_registration": "MinFirstDay"
        },inplace=True
    )
    days_from_regis['cutoffDay'] = days_from_regis.MinFirstDay + cutOffDay
    days_from_regis['lateUser'] = (days_from_regis['MinFirstDay'] > 0).astype(int)

    Engagement = df_events.merge(days_from_regis, on='UserID',how='left')
    Engagement = Engagement[
        (Engagement['cutoffDay'] - Engagement['nth_day_from_registration']) > 0
        ].reset_index(drop=True)
    Engagement['usedDays'] = Engagement['nth_day_from_registration'] - Engagement['MinFirstDay']
    return Engagement

def setLabel(df_events, df_Payment, markDay=30):
    df_events = df_events.merge(df_Payment[['UserID','gapday']], how='left', on='UserID')
    """
        If label = True -> User paid before markDay
        else -> Free user
    """
    df_events['label'] = ((df_events['gapday'] - df_events['MinFirstDay']) <= markDay).astype(int)

    return df_events


In [9]:
Engagement_df = getEvent_before_cutoff(df_Engagement, 3)
Engagement_df = setLabel(Engagement_df, df_Payment, 30)
Engagement_df

,EventDate,UserID,ActionType,UserJoinedDate,nth_day_from_registration,open_time,payment_screen_time,DeviceBrand,UsageTime,Records,lastuse,MinFirstDay,cutoffDay,lateUser,usedDays,gapday,label
0,2022-05-30,c65f5b3f-e220-4c2a-a94e-74c4749f0783,learn,2022-05-28 09:11:16.181000+07:00,2,1,0,samsung,33.80,26,2022-05-30 18:41:23+07:00,0,3,0,2,0.0,1
1,2022-05-18,a6947e28-6181-44b7-9370-9dbdcec2c72a,discovery,2022-05-18 11:39:00.331000+07:00,0,1,5,oppo,23.33,14,2022-05-18 21:48:51+07:00,0,3,0,0,NaN,0
2,2022-04-14,91236b2e-c4ef-4dea-b042-ee69996aaff6,entertainment,2022-04-13 22:04:52.084000+07:00,1,1,0,apple,0.72,1,2022-04-14 12:43:09+07:00,0,3,0,1,NaN,0
3,2023-03-17,16b5eb5a-460a-44fb-8c7f-60f98d5d93b2,learn,2023-03-17 20:47:49.044000+07:00,0,1,4,apple,40.17,34,2023-03-17 21:47:57+07:00,0,3,0,0,NaN,0
4,2022-07-01,8e19c46a-76ff-41aa-b3c9-91d860f2df07,learn,2022-07-01 20:21:09.145000+07:00,0,0,1,xiaomi,10.10,11,2022-07-01 20:38:08+07:00,0,3,0,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384347,2022-10-22,e24a3679-74b5-4dae-a257-1d46907f95fb,discovery,2022-10-22 12:11:35.368000+07:00,0,2,3,apple,6.90,2,2022-10-22 13:09:29+07:00,0,3,0,0,NaN,0
384348,2022-04-15,af0f6d8e-a686-494b-b636-8ba71931a766,learn,2022-04-15 12:08:30.030000+07:00,0,0,0,realme,3.27,5,2022-04-15 12:13:54+07:00,0,3,0,0,NaN,0
384349,2022-08-21,6fa05f01-4291-484c-8464-36a782585812,learn,2022-08-21 15:01:33.666000+07:00,0,0,0,samsung,1.10,1,2022-08-21 15:04:26+07:00,0,3,0,0,NaN,0
384350,2022-06-13,e89b6774-1914-4160-b612-76eb62a7285f,entertainment,2022-06-12 18:22:15.094000+07:00,1,0,0,NaN,7.33,3,2022-06-13 16:43:09+07:00,0,3,0,1,NaN,0


In [10]:
df_Payment[df_Payment.gapday.isna()]['gapday'].iloc[0] - 30 < 0

False

In [18]:
sample_ID = Engagement_df[
    (Engagement_df.lateUser == 1) &
    (Engagement_df.label == 1)
]['UserID'].iloc[2]

In [19]:
Engagement_df[Engagement_df.UserID == sample_ID].sort_values(by='usedDays')

,EventDate,UserID,ActionType,UserJoinedDate,nth_day_from_registration,open_time,payment_screen_time,DeviceBrand,UsageTime,Records,lastuse,MinFirstDay,cutoffDay,lateUser,usedDays,gapday,label
234979,2022-06-18,bbc3f467-29e5-49d3-97ec-6d5de3a67ed8,learn,2022-06-17 22:43:48.271000+07:00,1,1,0,apple,14.15,15,2022-06-18 16:10:40+07:00,1,4,1,0,1.0,1
206,2022-06-19,bbc3f467-29e5-49d3-97ec-6d5de3a67ed8,learn,2022-06-17 22:43:48.271000+07:00,2,0,0,NaN,3.72,4,2022-06-19 07:44:08+07:00,1,4,1,1,1.0,1
299450,2022-06-19,bbc3f467-29e5-49d3-97ec-6d5de3a67ed8,entertainment,2022-06-17 22:43:48.271000+07:00,2,0,0,NaN,0.23,1,2022-06-19 07:39:27+07:00,1,4,1,1,1.0,1
